In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo-1106",
    temperature=0.1,  # 창의적인 정도, 0~1 : 1에 가까울수록 창의적이고 무작위성을 가짐
    # max_tokens=5000,
)

In [2]:
# OutputParser & LangChain expression language(표현언어)
# LangChain expression language는 코드를 많이 줄여 줄 수 있음
# 또한, 다양한 template과 LLM 호출, 그리고 서로 다른 응답(response)을 함께 사용하게 해줌 -> Orchestration : 오케스트레이션

# OutputParser가 필요한 이유 -> LLM의 응답(response)을 변형해야 할 때가 있기 때문

In [25]:
# OutputParser

from langchain.schema import BaseOutputParser


class CommaOutputParser(BaseOutputParser):
    # def parse(self, text):
    #     # OutputParser는 항상 parse 메서드가 필요함
    #     # text로 전체 텍스트를 입력받음
    #     return text.strip().split(",")
    #     # .strip()은 앞뒤 공백을 잘라내줌
    #     # , comma를 기준으로 잘라서 array로 만듦

    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))


p = CommaOutputParser()

# p.parse("Hello, how are you") # -> ['Hello', ' how are you']
# p.parse(" Hello, how, are, you ") # -> ['Hello', ' how', ' are', ' you'] : 전체 텍스트의 앞뒤 공백은 잘라지지만 ,로 분할된 부분에서는 공백이 그대로 있음
p.parse(
    " Hello, how, are, you "
)  # -> ['Hello', 'how', 'are', 'you'] : ,로 분할된 부분도 공백 잘라주기

['Hello', 'how', 'are', 'you']

In [20]:
# Template
template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase. Do NOT reply with anything else.",
        ),
        ("human", "{question}"),
    ]
)

# Prompt
prompt = template.format_messages(max_items=10, question="What are the colors?")

result = chat.predict_messages(
    prompt
)  # AIMessage(content='red, blue, green, yellow, orange, purple, pink, black, white, brown')

# OutputParser
p = CommaOutputParser()

p.parse(result.content)

['red',
 'blue',
 'green',
 'yellow',
 'orange',
 'purple',
 'pink',
 'black',
 'white',
 'brown']

In [21]:
# LangChain expression language
# chat model, OuputParser, template만 사용

template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase. Do NOT reply with anything else.",
        ),
        ("human", "{question}"),
    ]
)

In [26]:
# chain
# chain은 기본적으로 모든 요소들을 합쳐주는 역할을 함
# 처음에는 template를 입력하고, 그 다음으로는 '|' 기호를 사용할 거임

chain = template | chat | CommaOutputParser()  # | 로 실행단계를 연결해서 명시해주는 거

chain.invoke(
    {"max_items": 5, "question": "What are the pokemons?"}
)  # 실행 : 입력값 dict

# chain끼리, chain들과 outputparser 등도 | 로 연결할 수 있음

['pikachu', 'charmander', 'bulbasaur', 'squirtle', 'jigglypuff']